In [ ]:
'''🔹 PART 1  Advanced Filtering & Conditions
 1  Filter passengers with:
Age between 18 and 40
Fare > median fare
Gender = female

 2  Select passengers who:
Are NOT in 3rd class
Survived
Embarked from "S"

 3  Count how many passengers match these filters

🔹 PART 2  GroupBy Tricks
 4  Group by Pclass and Gender and calculate:
Average Age
Survival Rate (hint: .mean() on "Survived")
 5  Find the class with the highest fare variance
 6  Use .agg() to calculate multiple stats:
For Fare: min, max, mean, std grouped by Embarked

🔹 PART 3  Pivot Tables & Custom Columns
 7  Create a pivot table:
Rows → Pclass
Columns → Gender
Values → Mean Fare

 8  Create a new column "FareLevel":
If Fare < 10 → "Low"
If Fare between 1050 → "Medium"
If Fare > 50 → "High"

 9  Use .apply() to label "AgeGroup" as:
012 → Child
1319 → Teen
2059 → Adult
60+ → Senior

 10  Create a summary table:
Count of each AgeGroup by Gender and Survival status'''

In [10]:
import pandas as pd
url = 'https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv'
df = pd.read_csv(url)

In [ ]:
'Part 1'
median_fare = df['Fare'].mean()
mask = (df['Age'] < 40) & (df['Age'] > 18) & ( df['Fare'] > median_fare) & df['Sex'] == 'female'
filtered_passangers = df[mask]
print(filtered_passangers.head())


In [ ]:
mask = (df['Pclass'] != 1) & ( df['Survived'] == 1) & (df['Embarked'] == 'S')
filtered_passangers = df[mask]
print(filtered_passangers['PassengerId'].count())

In [13]:
'Part 2'
grouped_pclass_gender = df.groupby(['Pclass','Sex'])
average_age = grouped_pclass_gender['Age'].mean()
survival_rate = grouped_pclass_gender['Survived'].mean() * 100

In [ ]:
class_fare_variances = df.groupby('Pclass')['Fare'].var()
highest_fare = class_fare_variances.idxmax()
print(highest_fare)

In [ ]:
fare_stats = df.groupby('Embarked')['Fare'].agg(
    min_fare = 'min',
    max_fare = 'max',
    mean_fare = 'mean',
    std_fare = 'std')

print(fare_stats)    

In [ ]:
df = df.rename(columns = {'Sex': 'Gender'}) 
pivot_table = df.pivot_table(values = 'Fare', index = 'Pclass', columns = 'Gender',aggfunc = 'mean')
print(pivot_table)

In [ ]:
import numpy as np
conditions = [
    df['Fare'] < 10,
    (df['Fare'] >= 10) & (df['Fare'] <= 50),
    df['Fare'] > 50
]
choices = ['Low','Medium','High']
df['FareLevel'] = np.select(conditions,choices,default = 'Unknown')

print(df[['Fare','FareLevel']].head())

In [ ]:
df['AgeGroup'] = df['Age'].apply(lambda age : 'Child' if age < 13 else
                                            'Not known'if pd.isna(age) else
                                            'Teen' if age < 20 else
                                            'Adult' if age < 60 else
                                            'Senior')
print(df[['Age','AgeGroup']].sample(n = 5))

In [ ]:
table_AgeGroup = df.groupby(['AgeGroup','Gender','Survived']).size().unstack(fill_value=0)
print(table_AgeGroup)